In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import os
#print(os.getcwd())


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nn_train.csv')
data.head()

,ID,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,...,Feature_1017,Feature_1018,Feature_1019,Feature_1020,Feature_1021,Feature_1022,Feature_1023,Feature_1024,binary_label,class_label
0,1,81,109,91,96,107,97,103,95,100,...,114,93,91,98,72,92,78,78,0,5
1,2,168,168,169,169,170,171,172,172,175,...,44,44,44,44,44,44,44,44,0,9
2,3,14,20,32,6,27,6,5,34,15,...,63,64,63,60,54,48,43,41,0,5
3,4,60,109,157,126,82,80,70,58,37,...,105,102,99,98,100,105,111,114,1,8
4,5,63,72,86,75,93,77,75,67,62,...,108,109,102,106,96,91,102,89,0,5


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
x = data.iloc[:,1:1025]

x = (x - x.mean())/(x.max() - x.min())

print(x.head())

   Feature_1  Feature_2  Feature_3  Feature_4  Feature_5  Feature_6  \
0  -0.136290  -0.028474  -0.104216  -0.089039  -0.050071  -0.093089   
1   0.204887   0.202898   0.201666   0.197235   0.196988   0.197107   
2  -0.399035  -0.377494  -0.335589  -0.441980  -0.363797  -0.449952   
3  -0.218643  -0.028474   0.154607   0.028608  -0.148110  -0.159756   
4  -0.206878  -0.173572  -0.123824  -0.171392  -0.104973  -0.171520   

   Feature_7  Feature_8  Feature_9  Feature_10  ...  Feature_1015  \
0  -0.073127  -0.107338  -0.090555   -0.096532  ...     -0.025994   
1   0.197461   0.194623   0.203563    0.201507  ...     -0.280896   
2  -0.457441  -0.346554  -0.423888   -0.374964  ...     -0.210308   
3  -0.202539  -0.252436  -0.337614   -0.245552  ...      0.024986   
4  -0.182931  -0.217142  -0.239575   -0.229866  ...      0.017143   

   Feature_1016  Feature_1017  Feature_1018  Feature_1019  Feature_1020  \
0     -0.079296     -0.002756     -0.082761     -0.087940     -0.057893   
1     -0

In [ ]:
y = data.iloc[:,-1]
print(y.head())

0    5
1    9
2    5
3    8
4    5
Name: class_label, dtype: int64


In [ ]:
x_train = x.to_numpy()
print(x_train.shape)
y_train = y.to_numpy()

print(type(y_train))
m = x_train.shape[0]
print((y_train.shape))
n = int(0.01*m)
x_test = x_train[:n,:]
x_train = x_train[n:,:]
print(x_train.shape)

y_test = y_train[:n]
y_train = y_train[n:]
print(y_train.shape)

(80000, 1024)
<class 'numpy.ndarray'>
(80000,)
(79200, 1024)
(79200,)


In [ ]:

class KNN:
   def __init__(self,k):
    self.k = k
   def fit(self,x,y):
    self.x = x
    self.y = y
   def dist(self,x1,x2):
    return np.sqrt(np.sum((x1-x2)**2))
   def predict(self,X):
    prediction = []
    i = 0
    for x in X:
        distance = [self.dist(x,x_train) for x_train in self.x]
        k_indices = np.argsort(distance)[:self.k]
        knearest =  [self.y[i] for i in(k_indices)]
        i = i+ 1
        print(i)

        counts = {}
        for num in (knearest):
          counts[num] = counts.get(num,0) + 1
        max_freq = max(counts, key=counts.get)
        prediction.append(max_freq)

    return np.array(prediction)

In [ ]:
k = 9
knn = KNN(k)
knn.fit(x_train, y_train)

# Predict
y_pred = knn.predict(x_test)
print("Predictions:", y_pred)
print(y_test)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
acc = np.mean(y_pred == y_test)
print("Accuracy:", (acc*100))
print(y_pred.shape)
#ant("accuracy On cross validation set : ",accuracy * 100)

def f1_score_avg(y,y_test,i,f1_score):
    p = 0
    r = 0

    if i > len(f1_score):
        return np.mean(f1_score)
    else:
        p = np.sum((y == i) & (y_test == i))/(np.sum((y == i)))# & (y_test != i)) + np.sum((y == i) & (y_test == i)))
        r = np.sum((y == i) & (y_test == i))/(np.sum((y_test == i)))# + np.sum((y == i) & (y_test == i)))
        f1_score[i-1] = 2*(p*r)/(p + r)

        return f1_score_avg(y,y_test,i+1,f1_score)
f1_score = np.zeros((10))
i = 1
f1_avg = f1_score_avg(y_pred,y_test,i ,f1_score)
print(f1_avg)


Accuracy: 38.375
(800,)
0.35172577966235863
